## Imports

In [35]:
import re
import sys
from datetime import timedelta
from os.path import abspath, join as opj
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from scipy.interpolate import interp1d
from scipy.spatial.distance import cdist
from scipy.stats import entropy, pearsonr
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.notebook import tqdm

sys.path.insert(0, abspath('../helpers/'))
from participant import Participant
from experiment import Experiment

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
%matplotlib inline

## Set paths

In [2]:
datadir = '../../data/'
rawdir = opj(datadir, 'raw')
figdir = '../../figures/'

## Load data

In [3]:
# four forces lecture transcript
with open(opj(rawdir, 'forces_transcript_timestamped.txt'), 'r') as f:
    ff_transcript = f.read()

# birth of stars lecture transcript
with open(opj(rawdir, 'bos_transcript_timestamped.txt'), 'r') as f:
    bos_transcript = f.read()
    
# questions
questions_df = pd.read_csv(opj(rawdir, 'questions.tsv'), sep='\t', 
                           names=['index', 'lecture', 'question', 
                                  'ans_A', 'ans_B', 'ans_C', 'ans_D'], 
                           index_col='index')

# participants' scores
raw_data = raw_data = pd.read_csv(opj(rawdir, 'psiturk-data-raw.tsv'), sep='\t', index_col='Unnamed: 0')
# scores_df = pd.read_csv(opj(rawdir, 'Graded_results_19f_49.csv'), index_col='Unnamed: 0')
participants = []
for ix, row in raw_data.iterrows():
    p = Participant.from_psiturk(row, subid=f'P{ix + 1}')
    participants.append(p)

## Set basic params

In [4]:
stop_words = stopwords.words('english') + ["let", "let's", "they'd", "they're", 
                                           "they've", "they'll", "that's", 
                                           "I'll", "I'm", "really", "even"]
# cv_params = {
#     'max_df': 0.95,
#     'min_df': 2,
#     'max_features': 500,
#     'stop_words': stop_words
# }

cv_params = {
    'strip_accents': 'unicode',
    'stop_words': stop_words,
}


lda_params = {
    'learning_method': 'batch',
    'random_state': 0
}

## Define some functions

In [5]:
def r2z(r):
    with np.errstate(invalid='ignore', divide='ignore'):
        return 0.5 * (np.log(1 + r) - np.log(1 - r))
    

def z2r(z):
    with np.errstate(invalid='ignore', divide='ignore'):
        return (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1)
    

def corr_mean(rs, axis=0):
    return z2r(np.nanmean([r2z(r) for r in rs], axis=axis))

In [6]:
def format_text(windows, sw=stop_words):
    clean_text = []
    for chunk in windows:
        no_punc = re.sub("[^a-zA-Z\s'-]+", '', chunk.lower()).replace('-', ' ')
        no_stop = ' '.join([word for word in no_punc.split() if word not in sw])
        clean = re.sub("'+", '', no_stop)
        clean_text.append(clean)
    return clean_text

In [7]:
def _ts_to_secs(ts):
    mins, secs = ts.split(':')
    mins, secs = int(mins), int(secs)
    return timedelta(minutes=mins, seconds=secs).total_seconds()
    

def parse_windows(transcript, wsize):
    lines = transcript.splitlines()
    text_lines = [l for ix, l in enumerate(lines) if ix % 2]
    ts_lines = [_ts_to_secs(l) for ix, l in enumerate(lines) if not ix % 2]    
    windows = []
    timestamps = []
    for ix in range(1, wsize):
        start, end = 0, ix
        windows.append(' '.join(text_lines[start : end]))
        timestamps.append((ts_lines[start] + ts_lines[end - 1]) / 2)

    for ix in range(len(ts_lines)):
        start = ix
        end = ix + wsize if ix + wsize <= len(text_lines) else len(text_lines)
        windows.append(' '.join(text_lines[start : end]))
        timestamps.append((ts_lines[start] + ts_lines[end - 1]) / 2)
        
    return windows, timestamps

In [8]:
def create_qa_corpus(df, key):
    grouped = df.groupby('lecture')
    gen_qs, ff_qs, bos_qs = grouped['question'].apply(format_text)
    answers_fmt = grouped['ans_A', 'ans_B', 'ans_C', 'ans_D'].agg(format_text)
    gen_correct, ff_correct, bos_correct = answers_fmt['ans_A']
    gen_ans, ff_ans, bos_ans = answers_fmt.apply(np.concatenate, axis=1).apply(list)
    
    # all formatted text for transforming
    all_text = {
        'ff_qs': ff_qs,
        'bos_qs': bos_qs,
        'gen_qs': gen_qs,
        'ff_ans': ff_ans,
        'bos_ans': bos_ans,
        'gen_ans': gen_ans
    }
    # subset of text used to fit model
    corpus_text = {k: v for k, v in all_text.items()}
    if 'q' in key:
        corpus_text['gen_qs'] = []
        corpus_text['gen_ans'] = []
    elif 'q' not in key.lower():
        corpus_text['ff_qs'] = []
        corpus_text['bos_qs'] = []
        corpus_text['gen_qs'] = []
    
    if 'a' in key:
        corpus_text['ff_ans'] = ff_correct
        corpus_text['bos_ans'] = bos_correct
        if 'Q' in key:
            corpus_text['gen_ans'] = gen_correct
    elif 'a' not in key.lower():
        corpus_text['ff_ans'] = []
        corpus_text['bos_ans'] = []
        corpus_text['gen_ans'] = []
        
    return corpus_text, all_text

In [9]:
def interp_lecture(lec_traj, timestamps):
    new_tpts = np.arange(timestamps[-1])
    interp_func = interp1d(timestamps, lec_traj, axis=0)
    return interp_func(new_tpts)

In [10]:
def model_content(n_top, wsize, corpus):
    # get text windows, timestamps
    ff_windows, ff_timestamps = parse_windows(ff_transcript, wsize)
    bos_windows, bos_timestamps = parse_windows(bos_transcript, wsize)
    
    # format lecture, question, answer text
    ff_lec = format_text(ff_windows)
    bos_lec = format_text(bos_windows)
    corpus_text, all_text = create_qa_corpus(questions_df, corpus)
    corpus_text['ff_lec'] = all_text['ff_lec'] = ff_lec
    corpus_text['bos_lec'] = all_text['bos_lec'] = bos_lec

    # set topic model params
    tm_params = {opt: val for opt, val in lda_params.items()}
    tm_params['n_components'] = n_top
    
    full_corpus = [doc for samples in corpus_text.values() for doc in samples]
    
    vecs = {}
    # fit CountVectorizer model
    tf_vectorizer = CountVectorizer(**cv_params)
    tf_vectorizer.fit(full_corpus)
    vecs['full_corpus'] = tf_vectorizer.transform(full_corpus)
    # transform lectures, questions, answers
    for samples, docs in all_text.items():
        vecs[samples] = tf_vectorizer.transform(docs)
        
    trajs = {}
    # fit LDA model
    topic_model = LatentDirichletAllocation(**tm_params)
    topic_model.fit(vecs['full_corpus'])
    # transform lectures, questions, answers
    for samples, docs in all_text.items():
        trajs[samples] = topic_model.transform(vecs[samples])
            
    # interpolate lectures to 1 sample per second
    trajs['ff_lec'] = interp_lecture(trajs['ff_lec'], ff_timestamps)
    trajs['bos_lec'] = interp_lecture(trajs['bos_lec'], bos_timestamps)

    return trajs

In [11]:
def symmetric_KL(a, b, c=.00000000001):
    return np.divide(entropy(a + c, b + c) + entropy(b + c, a + c), 2)
    
def reconstruct_trace(video_model, questions_model, acc):
    wz = 1 - cdist(video_model, questions_model, metric=symmetric_KL)
    wz -= np.min(wz)
    wz /= np.max(wz)
    a = np.sum(wz, axis=1)
    b = np.sum(wz[:, list(map(bool, acc))], axis=1)
    ba = np.array(np.divide(b, a), ndmin=2)
    return video_model * ba.T

In [12]:
def score_params(traj_dict):
    ff_lec = traj_dict['ff_lec']
    bos_lec = traj_dict['bos_lec']
    ff_bos_questions = np.concatenate((traj_dict['ff_qs'], traj_dict['bos_qs']))
#     ff_bos_scores = scores.loc[scores.video != 0]
    p_wrong = []
    msds = []
    
#     for subid in tqdm(ff_bos_scores.participantID.unique(), desc='subjects', leave=False):
#         sub_pwrong = []
#         sub_msds = []
#         for lec_ix, lec in enumerate([ff_lec, bos_lec], start=1):
#             sub_scores = ff_bos_scores.loc[(ff_bos_scores.video == lec_ix) 
#                                            & (ff_bos_scores.participantID == subid) 
#                                            & (ff_bos_scores.set == lec_ix)]
            
            
    for sub in participants:
        sub_pwrong = []
        sub_msds = []
        for lec_ix, lec in enumerate([ff_lec, bos_lec], start=1):
            sub_scores = sub.get_data(qset=lec_ix, lecture=lec_ix)
            # compute p_wrong
            sub_acc = sub_scores['accuracy'].tolist()
            sub_pc = sum(sub_acc) / len(sub_acc)
            sub_pwrong.append(1 - sub_pc)
            # reconstruct trace
            qs_ix = sub_scores['qID'] - 1
            q_vecs = ff_bos_questions[qs_ix]
            sub_trace = reconstruct_trace(lec, q_vecs, sub_acc)
            # compute mean squared euclidean distance
            sub_msd = np.mean(np.diag(cdist(lec, sub_trace)) ** 2)
            sub_msds.append(sub_msd)
        p_wrong.append(np.mean(sub_pwrong))
        msds.append(np.mean(sub_msds))
            
    return pearsonr(p_wrong, msds)[0]

In [13]:
def grid_search(gs_params):
    n_top, wsize, corpus = gs_params
    trajs = model_content(n_top, wsize, corpus)
    corr = score_params(trajs)
    print(f'\t{gs_params} : {corr}')
    return corr

## Grid search params

In [14]:
# (maximum) number of topics to fit the topic model
n_topics = [20, 25, 30, 35, 40, 50, 100]
# length of lecture transcript sliding window
lecture_wsize = [10, 15, 20, 25, 30, 35, 40]
# training corpus for topic model
# - l: lecture sliding windows
# - q: lecture-related multiple choice questions' text
# - Q: all multiple choice questions' text
# - a: correct responses to multiple choice questions
# - A: all options for responses to multiple choice questions
corpora = ['l', 'lq', 'lQ', 'lqa', 'lQa', 'lqA', 'lQA']

param_grid = [(t, w, c) for t in n_topics for w in lecture_wsize for c in corpora]

In [15]:
param_scores = []
for params in tqdm(param_grid, desc='params', leave=True):
    result = grid_search(params)
    param_scores.append(result)

	(20, 10, 'l') : 0.9292345805367535
	(20, 10, 'lq') : 0.9326575959894322
	(20, 10, 'lQ') : 0.9501855962868262
	(20, 10, 'lqa') : 0.9375475192764581
	(20, 10, 'lQa') : 0.9494911264696513
	(20, 10, 'lqA') : 0.9462571515340038
	(20, 10, 'lQA') : 0.9338328775888544
	(20, 15, 'l') : 0.9149752808396528
	(20, 15, 'lq') : 0.9300314043219959
	(20, 15, 'lQ') : 0.9497618196214459
	(20, 15, 'lqa') : 0.9551018264214164
	(20, 15, 'lQa') : 0.9393481458409171
	(20, 15, 'lqA') : 0.9460824413166946
	(20, 15, 'lQA') : 0.9357751203454359
	(20, 20, 'l') : 0.9200625474181345
	(20, 20, 'lq') : 0.9599887266478103
	(20, 20, 'lQ') : 0.9407000550341482
	(20, 20, 'lqa') : 0.9493457883323438
	(20, 20, 'lQa') : 0.9398856425481943
	(20, 20, 'lqA') : 0.9438852624518757
	(20, 20, 'lQA') : 0.9463770462721569
	(20, 25, 'l') : 0.9186802325106981
	(20, 25, 'lq') : 0.9451604595088149
	(20, 25, 'lQ') : 0.9457461529811974
	(20, 25, 'lqa') : 0.9484628852188567
	(20, 25, 'lQa') : 0.9399173161316285
	(20, 25, 'lqA') : 0.9513745

	(40, 25, 'lqa') : 0.9104665054555309
	(40, 25, 'lQa') : 0.9067696708480557
	(40, 25, 'lqA') : 0.9376287346290496
	(40, 25, 'lQA') : 0.9171584694577044
	(40, 30, 'l') : 0.9306883557852025
	(40, 30, 'lq') : 0.921749835252092
	(40, 30, 'lQ') : 0.9253637754956353
	(40, 30, 'lqa') : 0.9398801778839188
	(40, 30, 'lQa') : 0.9098362653849843
	(40, 30, 'lqA') : 0.9049284022548137
	(40, 30, 'lQA') : 0.917552108958675
	(40, 35, 'l') : 0.9324893410401003
	(40, 35, 'lq') : 0.9315271156321495
	(40, 35, 'lQ') : 0.9166256507360084
	(40, 35, 'lqa') : 0.90812120564549
	(40, 35, 'lQa') : 0.9070267090206192
	(40, 35, 'lqA') : 0.9197597812112416
	(40, 35, 'lQA') : 0.8934863104507783
	(40, 40, 'l') : 0.9305577577116528
	(40, 40, 'lq') : 0.9267149268217695
	(40, 40, 'lQ') : 0.9314172884855134
	(40, 40, 'lqa') : 0.9252390935743924
	(40, 40, 'lQa') : 0.9185084038688005
	(40, 40, 'lqA') : 0.9024935797559858
	(40, 40, 'lQA') : 0.9051417323830513
	(50, 10, 'l') : 0.9186406775254272
	(50, 10, 'lq') : 0.9243722885

In [20]:
np.array(param_grid)[np.argmax(param_scores)]

array(['25', '15', 'lQ'], dtype='<U21')

In [41]:
gs_results = pd.DataFrame(param_grid, columns=['topics', 'wsize', 'corpus'])
gs_results['correlation'] = param_scores
# gs_results.to_pickle(opj(datadir, 'grid_search_results.p'))

In [22]:
# gs_results = pd.read_pickle(opj(datadir, 'grid_search_results.p'))

In [23]:
gs_results

,topics,wsize,corpus,correlation
0,20,10,l,0.929235
1,20,10,lq,0.932658
2,20,10,lQ,0.950186
3,20,10,lqa,0.937548
4,20,10,lQa,0.949491
...,...,...,...,...
338,100,40,lQ,0.880405
339,100,40,lqa,0.889668
340,100,40,lQa,0.924142
341,100,40,lqA,0.908181


In [24]:
# param_scores = gs_results['correlation'].tolist()

In [40]:
fig, axarr = plt.subplots(2, 4)
fig.set_size_inches(14, 6)
axarr = axarr.flatten()

param_arr = np.array(param_scores).reshape(7, 7, 7)

for i, (ax, arr) in enumerate(zip(axarr, param_arr.T)):
    cb = True
    yt = lecture_wsize if i in [0, 4] else []
    ax = sns.heatmap(arr, vmin=.85, vmax=1, xticklabels=n_topics, yticklabels=yt, ax=ax, cbar=cb)
    ax.set_title(f'Corpus: {corpora[i]}')
    if i >= 3:
        ax.set_xlabel('Number of topics')
    if i in [0, 4]:
        ax.set_ylabel('Lecture window size')
        
    if i == 2:
        max_cell = np.unravel_index(arr.argmax(), arr.shape)
        ax.add_patch(Rectangle(max_cell, 1, 1, fill=False, edgecolor='#FFF9AE', lw=3))
        
    
    
axarr[-1].axis('off')
plt.tight_layout()
plt.savefig(opj(figdir, 'grid_search.pdf'), bbox_inches='tight')
plt.show()